## 다중 분류 문제
- 난방 부하 단계를 예측해주세요!

- 예측할 값(y): Heat_Load (Very Low, Low, Medium, High, Very High)
- 평가: f1-macro
- data: train.csv, test.csv
- 제출 형식: result.csv파일을 아래와 같은 형식으로 제출
~~~
pred
Very Low
Low
High
...
Very High
~~~

### 답안 제출 참고
- pd.read_csv('result.csv') 로 제출 코드 확인

In [39]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# print(train.shape, test.shape)
# (537, 10) (231, 9)

# print(train.head())
#    Compac  Surf_Area  Wall_Area    Roof Height Orient  Glaze_Area  \
# 0    0.74      686.0      245.0   220.5  Short  South        0.25   
# 1    0.98      514.5      294.0   Small   Tall  South        0.40   
# 2    0.90      563.5      318.5  Medium   Tall   West        0.25   
# 3    0.74      686.0      245.0   220.5  Short  South        0.25   
# 4    0.98      514.5      294.0   Small   Tall   East        0.25   

#    Glaze_Distr  Cool_Load Heat_Load  
# 0            3      14.72  Very Low  
# 1            2      33.94      High  
# 2            1      37.58      High  
# 3            2      15.18  Very Low  
# 4            2      29.69      High  
# print(test.head())
#    Compac  Surf_Area  Wall_Area    Roof Height Orient  Glaze_Area  \
# 0    0.64      784.0      343.0   220.5  Short  South        0.40   
# 1    0.82      612.5      318.5   Large   Tall  North        0.40   
# 2    0.76      661.5      416.5  Medium   Tall  South        0.10   
# 3    0.74      686.0      245.0   220.5  Short   East        0.40   
# 4    0.64      784.0      343.0   220.5  Short  North        0.25   

#    Glaze_Distr  Cool_Load  
# 0            4      22.25  
# 1            3      32.43  
# 2            5      33.64  
# 3            1      17.25  
# 4            5      20.13  
# print(train.info())
#  0   Compac       537 non-null    float64
#  1   Surf_Area    537 non-null    float64
#  2   Wall_Area    537 non-null    float64
#  3   Roof         537 non-null    object 
#  4   Height       537 non-null    object 
#  5   Orient       537 non-null    object 
#  6   Glaze_Area   537 non-null    float64
#  7   Glaze_Distr  537 non-null    int64  
#  8   Cool_Load    537 non-null    float64
#  9   Heat_Load    537 non-null    object 
# print(train.describe())
#           Compac   Surf_Area   Wall_Area  Glaze_Area  Glaze_Distr   Cool_Load
# count  537.00000  537.000000  537.000000  537.000000   537.000000  537.000000
# mean     0.76879  667.796089  318.271881    0.237244     2.808194   24.939628
# std      0.10636   88.123635   42.447824    0.132183     1.541251    9.600170
# min      0.62000  514.500000  245.000000    0.000000     0.000000   11.170000
# 25%      0.69000  588.000000  294.000000    0.100000     2.000000   15.770000
# 50%      0.76000  661.500000  318.500000    0.250000     3.000000   25.160000
# 75%      0.86000  735.000000  343.000000    0.400000     4.000000   33.540000
# max      0.98000  808.500000  416.500000    0.400000     5.000000   48.030000
# print(train.describe(include="O"))
#          Roof Height Orient Heat_Load
# count     537    537    537       537
# unique      4      2      4         5
# top     220.5   Tall  South  Very Low
# freq      257    280    145       142

# print(test.isnull().sum()) -> 없음

# 전처리
# 타겟 분류
target = train.pop('Heat_Load')
# print(target) -> 문제 없음
# print(train.shape, test.shape)
# (537, 9) (231, 9)

# 1. 베이스 라인 F1:  0.9574016121384542 정확도:  0.9629629629629629
n_cols = train.select_dtypes(exclude="O").columns

# # 2. 범주형 선택 F1:  0.9438476603693994 정확도:  0.9506172839506173
# o_cols = train.select_dtypes(include="O").columns
# from sklearn.preprocessing import LabelEncoder, StandardScaler

# for col in o_cols:
#     le = LabelEncoder()
#     train[col] = le.fit_transform(train[col])
#     test[col] = le.transform(test[col])
    
# # 3. 정규화 F1:  0.9438476603693994 정확도:  0.9506172839506173
# scaler = StandardScaler()
# train[n_cols] = scaler.fit_transform(train[n_cols])
# test[n_cols] = scaler.transform(test[n_cols])

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train[n_cols], target, test_size=0.15, random_state=2022)

# 4. 하이퍼 파라미터 튜닝 F1:  0.9732572391395922 정확도:  0.9753086419753086
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
model = RandomForestClassifier(n_estimators=300, random_state=2022)
model.fit(X_tr, y_tr)
pred = model.predict(X_val)
print('F1: ', f1_score(y_val, pred, average='macro'))
print('정확도: ', accuracy_score(y_val, pred))
pred = model.predict(test[n_cols])
submit = pd.DataFrame(
    {
        'pred': pred
    }
)
submit.to_csv('작업형2제출.csv', index=False)


F1:  0.9732572391395922
정확도:  0.9753086419753086


In [41]:
df1 = pd.read_csv('작업형2제출.csv')
df1

,pred
0,Low
1,High
2,High
3,Low
4,Low
...,...
226,Very Low
227,Medium
228,Very Low
229,Low
